# Wine Dataset Analysis

## 1. 데이터셋 설명 (Dataset Description)

이 노트북에서는 scikit-learn에 내장된 **Wine Dataset**을 분석합니다.
데이터셋은 이탈리아의 같은 지역에서 재배된 세 가지 다른 품종의 와인에 대한 화학적 분석 결과를 포함하고 있습니다.

- **샘플 수 (Samples)**: 178개
- **특성 수 (Features)**: 13개 (Alcohol, Malic acid, Ash, Alcalinity of ash, Magnesium, Total phenols, Flavanoids, Nonflavanoid phenols, Proanthocyanins, Color intensity, Hue, OD280/OD315 of diluted wines, Proline)
- **클래스 (Target)**: 3개 (class_0, class_1, class_2)

데이터를 분석(EDA)하고, 전처리 및 특성 공학을 거쳐 6가지 모델을 학습시킨 후, 상위 4개 모델로 앙상블 모델을 구축하고 하이퍼파라미터 튜닝을 수행합니다.

### 라이브러리 임포트 (Library Import)
데이터 분석과 머신러닝 모델링에 필요한 주요 라이브러리들을 불러옵니다.
- **pandas, numpy**: 데이터 구조 처리 및 수치 연산
- **matplotlib, seaborn**: 데이터 시각화
- **sklearn**: 머신러닝 모델, 데이터셋 로드, 전처리, 평가 지표 등

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 시각화 설정
plt.style.use('seaborn-v0_8')
sns.set(font_scale=1.2)

## 2. 데이터 로드 및 EDA (Exploratory Data Analysis)

### 데이터 로드 및 데이터프레임 변환
`load_wine()` 함수를 통해 와인 데이터셋을 로드합니다. 데이터 분석의 편의를 위해 `features`(문제 데이터)와 `target`(정답 데이터)을 결합하여 하나의 `pandas DataFrame`으로 변환합니다.

In [ ]:
# 데이터 로드
wine = load_wine()
df = pd.DataFrame(data=wine.data, columns=wine.feature_names)
df['target'] = wine.target

# 데이터 확인
print("데이터 크기:", df.shape)
display(df.head())

### 데이터 기본 정보 및 기초 통계량 확인
- **`info()`**: 데이터의 타입, 결측치(Null) 여부, 전체 데이터 개수 등을 확인하여 데이터의 전반적인 구조를 파악합니다.
- **`describe()`**: 각 수치형 변수의 평균, 표준편차, 4분위수 등을 확인하여 데이터의 분포와 이상치 존재 가능성을 짐작해봅니다.

In [ ]:
# 데이터 기본 정보 및 통계량
display(df.info())
display(df.describe())

### 타겟 클래스 분포 시각화
와인의 품종(class 0, 1, 2)별 데이터 개수를 막대 그래프로 확인합니다. 특정 클래스의 데이터가 너무 적거나 많으면(불균형 데이터) 모델 학습에 영향을 줄 수 있으므로 확인이 필요합니다.

In [ ]:
# 타겟 클래스 분포 확인
plt.figure(figsize=(6, 4))
sns.countplot(x='target', data=df)
plt.title('Target Class Distribution')
plt.show()

### 상관관계 분석 (Correlation Analysis)
특성(Feature)들 간의 상관관계를 히트맵(Heatmap)으로 시각화합니다.
- 1에 가까울수록 양의 상관관계(하나가 오르면 다른 하나도 오름)
- -1에 가까울수록 음의 상관관계(하나가 오르면 다른 하나는 내림)
- 타겟 변수와 상관관계가 높은 특성이 무엇인지, 혹은 특성끼리 너무 강한 상관관계를 보이지는 않는지(다중공선성) 확인합니다.

In [ ]:
# 상관관계 히트맵
plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(), annot=True, fmt='.2f', cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

## 3. 데이터 전처리 및 특성 엔지니어링 (Preprocessing & Feature Engineering)

### 데이터 분할 및 스케일링
모델 학습을 위해 데이터를 가공합니다.
1. **데이터 분할 (`train_test_split`)**: 전체 데이터를 학습용(Train)과 테스트용(Test)으로 나눕니다. 모델이 학습하지 않은 데이터에 대해서도 잘 예측하는지 평가하기 위함입니다.
2. **스케일링 (`StandardScaler`)**: 각 특성마다 값의 범위가 다르므로(예: 어떤 건 0.1~0.5, 어떤 건 100~1000), 이를 비슷한 범위(평균 0, 분산 1)로 맞춰줍니다. 특히 SVM이나 KNN 같은 거리 기반 모델에서는 필수적입니다.

In [ ]:
# 특성(X)과 타겟(y) 분리
X = df.drop('target', axis=1)
y = df['target']

# 학습용과 테스트용 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 스케일링 (StandardScaler)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Train Data Shape:", X_train_scaled.shape)
print("Test Data Shape:", X_test_scaled.shape)

## 4. 모델링 (Modeling - 6 Models)

### 다양한 머신러닝 모델 학습 및 비교
대표적인 분류 모델 6가지를 선정하여 학습시키고 정확도를 비교합니다.
- 로지스틱 회귀, 결정 트리, 랜덤 포레스트, SVM, KNN, 그라디언트 부스팅
- 각 모델을 학습(`fit`)시키고, 테스트 데이터로 점수(`score`)를 산출합니다.

In [ ]:
# 6가지 모델 정의
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVC': SVC(probability=True, random_state=42),
    'KNN': KNeighborsClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

# 모델 성능 평가 (Cross Validation)
model_scores = {}
print("Model Accuracy Scores:")
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    score = model.score(X_test_scaled, y_test)
    model_scores[name] = score
    print(f"{name}: {score:.4f}")

## 5. 앙상블 모델 및 하이퍼파라미터 튜닝 (Ensemble & Tuning)

### 상위 모델 선정, 튜닝 및 앙상블 구축
가장 성능이 좋았던 상위 4개 모델을 선택하여 더 강력한 모델을 만듭니다.
1. **GridSearchCV**: 각 모델별로 최적의 성능을 내는 하이퍼파라미터를 찾습니다.
2. **VotingClassifier (Soft Voting)**: 튜닝된 4개의 모델이 각자 예측한 확률을 평균 내어 최종 결과를 결정하는 앙상블 모델을 만듭니다. 단일 모델보다 과적합을 방지하고 일반화 성능을 높일 수 있습니다.

In [ ]:
# 성능이 좋은 상위 4개 모델 선정
sorted_models = sorted(model_scores.items(), key=lambda x: x[1], reverse=True)
top_4_models = sorted_models[:4]
print("Top 4 Models selected for Ensemble:", top_4_models)

# 상위 4개 모델 인스턴스 가져오기 (튜닝 전)
selected_model_names = [m[0] for m in top_4_models]

# 하이퍼파라미터 튜닝 (간단한 예시 범위 설정)
# 실제로는 각 모델별로 더 세밀한 그리드 탐색이 필요합니다.

tuned_estimators = []

for name in selected_model_names:
    model = models[name]
    print(f"\nTuning {name}...")
    
    if name == 'Random Forest':
        params = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20]}
    elif name == 'SVC':
        params = {'C': [0.1, 1, 10], 'kernel': ['rbf', 'linear']}
    elif name == 'Gradient Boosting':
        params = {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1]}
    elif name == 'KNN':
        params = {'n_neighbors': [3, 5, 7], 'weights': ['uniform', 'distance']}
    elif name == 'Logistic Regression':
        params = {'C': [0.1, 1, 10]}
    elif name == 'Decision Tree':
         params = {'max_depth': [None, 5, 10, 20]}
    else:
        params = {}
    
    if params:
        grid = GridSearchCV(model, params, cv=5, scoring='accuracy', n_jobs=-1)
        grid.fit(X_train_scaled, y_train)
        best_model = grid.best_estimator_
        print(f"Best Params for {name}: {grid.best_params_}")
    else:
        best_model = model
        best_model.fit(X_train_scaled, y_train)
    
    tuned_estimators.append((name, best_model))

# 앙상블 모델 생성 (VotingClassifier - Soft Voting)
ensemble_model = VotingClassifier(estimators=tuned_estimators, voting='soft')
ensemble_model.fit(X_train_scaled, y_train)
print("\nEnsemble Model Training Completed.")

## 6. 모델 평가 (Evaluation)

### 최종 성능 평가
완성된 앙상블 모델을 테스트 데이터로 평가합니다.
- **Accuracy**: 정답률
- **Classification Report**: 클래스별 Precision(정밀도), Recall(재현율), F1-score
- **Confusion Matrix**: 실제 클래스와 예측 클래스 간의 관계를 시각화하여 오분류 패턴 확인

In [ ]:
# 앙상블 모델 예측 및 평가
y_pred = ensemble_model.predict(X_test_scaled)

# 정확도
acc = accuracy_score(y_test, y_pred)
print(f"Ensemble Model Accuracy: {acc:.4f}\n")

# 분류 리포트
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=wine.target_names))

# 혼동 행렬 시각화
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=wine.target_names, yticklabels=wine.target_names)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()